<a href="https://colab.research.google.com/github/LeonardoSanBenitez/open-capivara-explainer/blob/main/notebooks/5.0%20-%20Finetuning%20with%20LoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Process:
1. Run the installation cell "Training dependencies setup"
2. Restart
3. Run all


In the guff conversion it says it needs restart agian, but worked without (I kept the guff setup separately because of dependency conflicts)

The downloaded models have to be put in the right folder afterwards, and maning adjusted

In [1]:
# Training dependencies setup
!pip install -q pandas peft==0.9.0 transformers==4.31.0 trl==0.4.7 bitsandbytes==0.43.0 accelerate scipy  tensorboardX

In [2]:
import os
from google.colab import files
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
from peft import get_peft_model

assert torch.cuda.is_available()
!nvidia-smi

Tue Aug 13 08:03:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# Model and tokenizer names
#base_model_name = "NousResearch/Llama-2-7b-chat-hf"
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#new_model_name = "llama-2-7b-enhanced" # Not used; You can give your own name for fine tuned model; will just be used to upload to HF?

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"


# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="cuda",
    #load_in_8bit=True,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# Data set
# mlabonne/guanaco-llama2-1k is a subset (1,000 samples) of the timdettmers/openassistant-guanaco
# human-generated, human-annotated, assistant-style conversation corpus
data_name = "mlabonne/guanaco-llama2-1k"
training_data = load_dataset(data_name, split="train")
print(training_data.shape)  # check the data
print(training_data[11])  # 11th is a QA sample in English

(1000, 1)
{'text': '<s>[INST] write me a 1000 words essay about deez nuts. [/INST] The Deez Nuts meme first gained popularity in 2015 on the social media platform Vine. The video featured a young man named Rodney Bullard, who recorded himself asking people if they had heard of a particular rapper. When they responded that they had not, he would respond with the phrase "Deez Nuts" and film their reactions. The video quickly went viral, and the phrase became a popular meme. \n\nSince then, Deez Nuts has been used in a variety of contexts to interrupt conversations, derail discussions, or simply add humor to a situation. It has been used in internet memes, in popular music, and even in politics. In the 2016 US presidential election, a 15-year-old boy named Brady Olson registered as an independent candidate under the name Deez Nuts. He gained some traction in the polls and even made appearances on national news programs.\n\nThe Deez Nuts meme has had a significant impact on popular culture

In [8]:
if not os.path.exists("/content/model_modified"):
  os.mkdir("/content/model_modified")

# Training Params
train_params = TrainingArguments(
    output_dir="/content/model_modified",
    num_train_epochs=1,
    per_device_train_batch_size=1,  # will run for 1k steps (because there is 1k examples at batch size 1)
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,  # every 50 steps
    logging_steps=50,
    learning_rate=4e-5,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, peft_parameters)
model.print_trainable_parameters()

# Trainer with LoRA configuration
device = torch.device("cuda:0")
base_model= base_model.to(device)
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,

)

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.10229075496156657


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [10]:
# Training
fine_tuning.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,1.850300
100,1.883300
150,1.773800
200,1.681300
250,1.568300
300,1.662000
350,1.684000
400,1.589400
450,1.568300
500,1.438700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=1000, training_loss=1.634079734802246, metrics={'train_runtime': 666.8898, 'train_samples_per_second': 1.499, 'train_steps_per_second': 1.499, 'total_flos': 2624983915143168.0, 'train_loss': 1.634079734802246, 'epoch': 1.0})

In [14]:
assert os.path.exists("/content/model_modified/checkpoint-1000")
!zip -r TinyLlama-1.1B-Chat-v1.0-LoRa-checkpoint-1000.zip /content/model_modified/checkpoint-1000
assert os.path.exists("TinyLlama-1.1B-Chat-v1.0-LoRa-checkpoint-1000.zip")

# Download model
files.download('TinyLlama-1.1B-Chat-v1.0-LoRa-checkpoint-1000.zip')

# Upload to HuggingFace
#trainer.push_to_hub()
#trainer.model.push_to_hub(training_args.output_dir)

  adding: content/model_modified/checkpoint-1000/ (stored 0%)
  adding: content/model_modified/checkpoint-1000/trainer_state.json (deflated 82%)
  adding: content/model_modified/checkpoint-1000/special_tokens_map.json (deflated 73%)
  adding: content/model_modified/checkpoint-1000/scheduler.pt (deflated 57%)
  adding: content/model_modified/checkpoint-1000/adapter_model.safetensors (deflated 8%)
  adding: content/model_modified/checkpoint-1000/tokenizer.model (deflated 55%)
  adding: content/model_modified/checkpoint-1000/training_args.bin (deflated 50%)
  adding: content/model_modified/checkpoint-1000/adapter_config.json (deflated 51%)
  adding: content/model_modified/checkpoint-1000/tokenizer_config.json (deflated 68%)
  adding: content/model_modified/checkpoint-1000/rng_state.pth (deflated 25%)
  adding: content/model_modified/checkpoint-1000/tokenizer.json (deflated 74%)
  adding: content/model_modified/checkpoint-1000/adapter_model.bin (deflated 8%)
  adding: content/model_modifie

# Inference

In [77]:
from peft import PeftModel
import os
from google.colab import files
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,

assert torch.cuda.is_available()

In [78]:
# Model and tokenizer names
base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="cuda",
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Merged
inference_model = PeftModel.from_pretrained(base_model, "/content/model_modified/checkpoint-1000")
device = torch.device("cuda:0")
inference_model= inference_model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
batch = llama_tokenizer('[INST] write me a 100 words essay about capibaras. [/INST]', return_tensors="pt")
batch = {k: v.to("cuda") for k, v in batch.items()}
output_tokens = inference_model.generate(
    **batch,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=llama_tokenizer.eos_token_id,
    pad_token_id=llama_tokenizer.pad_token_id,
)

output_tokens

In [ ]:
target_predicted = llama_tokenizer.decode(output_tokens[0], skip_special_tokens=False).split("<|endcontext|>")[0]
target_predicted

# Quantization and GUFF conversion

In [1]:
# Setup
import locale
locale.getpreferredencoding = "UTF-8"
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt

#!python llama.cpp/convert_lora_to_gguf.py -h
#!ls -l llama.cpp/convert*

Cloning into 'llama.cpp'...
remote: Enumerating objects: 32086, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 32086 (delta 67), reused 110 (delta 55), pack-reused 31953
Receiving objects: 100% (32086/32086), 57.52 MiB | 6.76 MiB/s, done.
Resolving deltas: 100% (22954/22954), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
  

In [79]:
# Saving the lora weights + base model (1~2 GBs)

# You can run the inference with both `inference_model` or `merged_model`
# Since the merge is oly needed for saving, i'm doing it here
merged_model = inference_model.merge_and_unload()
merged_model.save_pretrained("/content/model_merged", save_adapters=True, save_embedding_layers=True)
llama_tokenizer.save_pretrained("/content/model_merged")

!python llama.cpp/convert_hf_to_gguf.py "/content/model_merged" \
  --outfile "/content/model_modified_merged_quantized.gguf" \
  --outtype q8_0
assert os.path.exists("/content/model_modified_merged_quantized.gguf")

# This finish executing before the model is fully downloaded, so wait...
files.download('/content/model_modified_merged_quantized.gguf')

# Zipping reduce about 4% size, so whatever...
#!zip /content/model_modified_merged_quantized.gguf.zip /content/model_modified_merged_quantized.gguf
#!ls -lah /content/model_modified_merged_quantized.gguf.zip
#files.download('/content/model_modified_merged_quantized.gguf.zip')

# Another option is saving to your personal drive
# This is faster and more reliable, but you have to setup the drive connection
#from google.colab import drive
#drive.mount('/content/gdrive',force_remount=True)
#!cp "/content/model_modified_merged_quantized.gguf" "/content/gdrive/MyDrive/PATH_IN_YOUR_DRIVE/model_modified_merged_quantized.gguf"

# Maybe `gdown` is also an option
# https://stackoverflow.com/questions/49428332/how-to-download-large-files-like-weights-of-a-model-from-colaboratory

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Saving base model
from huggingface_hub import snapshot_download
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
snapshot_download(
    repo_id=model_id,
    local_dir="/content/model_base",
    local_dir_use_symlinks=False,
    revision="main",
  )

!python llama.cpp/convert_hf_to_gguf.py "/content/model_base" \
  --outfile "/content/model_base.gguf" \
  --outtype q8_0

assert os.path.exists("/content/model_base.gguf")
files.download('/content/model_base.gguf')

INFO:hf-to-gguf:Loading model: model_base
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,               torch.bfloat16 --> Q8_0, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {2048, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> Q8_0, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> Q8_0, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> Q8_0, shape = 

In [52]:
# Saving just the lora weights (a few MBs)
!python llama.cpp/convert_lora_to_gguf.py "/content/model_modified/checkpoint-1000" \
  --base "/content/model_base" \
  --outfile "/content/model_modified_quantized.gguf" \
  --outtype q8_0

assert os.path.exists("/content/model_modified_quantized.gguf")
files.download('/content/model_modified_quantized.gguf')

INFO:lora-to-gguf:Loading base model: model_base
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:lora-to-gguf:Exporting model...
INFO:hf-to-gguf:blk.0.attn_q.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.0.attn_q.weight.lora_b,  torch.float32 --> F16, shape = {8, 2048}
INFO:hf-to-gguf:blk.0.attn_v.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.0.attn_v.weight.lora_b,  torch.float32 --> F16, shape = {8, 256}
INFO:hf-to-gguf:blk.1.attn_q.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.1.attn_q.weight.lora_b,  torch.float32 --> F16, shape = {8, 2048}
INFO:hf-to-gguf:blk.1.attn_v.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.1.attn_v.weight.lora_b,  torch.float32 --> F16, shape = {8, 256}
INFO:hf-to-gguf:blk.10.attn_q.weight.lora_a, torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.10.attn_q.weight.lora_b, torch.float32 --> F16, 